In [93]:
# ! ls ../data

# Packages 

In [1]:
%load_ext autoreload
%autoreload 2
import sys

import logging
base_dir = '../'
sys.path.append(base_dir)
import os
from utils import *

import joblib
import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from gensim.similarities.annoy import AnnoyIndexer


from annoy import AnnoyIndex
import polars as pl
import implicit
from src.eval import model_eval

import scipy.sparse as sps
from utils import str2list
from src.config import raw_data_session_id_dir, candidate_file_name
from lightgbm import LGBMRanker
from lightgbm import early_stopping
from utils import *
from src.case_analysis import show_single_case
# try github 

/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


# Config 

In [2]:
debug = False

task = 'task2'

version = 'v3'
# # target locales: locales needed for task1
if task == 'task1':
    target_locals = ["DE", 'JP', 'UK']
elif task == 'task2':
    target_locals = ['ES', 'FR', 'IT']
else:
    assert 1 == 2


ranker_train_data_dir = f'../data/rank_train_data_{task}_{version}'

rank_model_version = f'{task}_rank_lgbm_{version}'

In [3]:
rank_model_version

'task2_rank_lgbm_v3'

In [4]:
! ls ../data/

candidates		   rank_train_data_task2_v2  session_item_features
eval_data		   rank_train_data_task2_v3  sub_files
rank_train_data		   rank_train_data_v2	     train_data
rank_train_data_task1_v10  raw_data
rank_train_data_task2	   raw_data_session_id


In [5]:

if debug:
    SAMPLE_NUM = 1000
else:
    SAMPLE_NUM = None

candidate_path = '../data/candidates/'
model_dir = '../model_training'


w2v_model_version = f'w2v_{task}'
nic_model_version = f'nic_{task}'

rank_model_dir = os.path.join(model_dir, rank_model_version)
model_for_eval = True
# w2v_topn=100
# nic_topn=100
# PREDS_PER_SESSION = 100

# num_tree = 100

# submit_file = f'submission_{task}_ALS.parquet'
num_tree = 100
w2v_model_dir = os.path.join(model_dir, w2v_model_version)
w2v_model_file = os.path.join(w2v_model_dir, f"{model_for_eval}.model")
annoy_index_file = os.path.join(w2v_model_dir, f"{str(num_tree)}_{model_for_eval}.index")


sub_file = f'../data/sub_files/{rank_model_version}.parque'
eval_sub_file = f'../data/sub_files/eval_{rank_model_version}.parque'

In [6]:
rank_model_dir

'../model_training/task2_rank_lgbm_v3'

In [7]:
! mkdir {rank_model_dir}

mkdir: cannot create directory ‘../model_training/task2_rank_lgbm_v3’: File exists


In [8]:
sub_file

'../data/sub_files/task2_rank_lgbm_v3.parque'

In [9]:
eval_sub_file

'../data/sub_files/eval_task2_rank_lgbm_v3.parque'

## Ranker config

In [12]:
estimator = 1000
early_stop_rounds = 10
eval_frac = 0.1
if debug:
    early_stop_rounds = 5
    
target = 'target'
feature_cols = [
    'next_item_weight'
    , 'next_item_weight_nfi'
    , 'next_item_weight_co_visit'
    , 'locale'
    , 'w2v_weight'
    , 'last_item_similarity'
    # , 'prev_item_similarity'	
    , 'prev_length'
    , 'price_mean_norm'	
    , 'price_last_norm'
    , 'price_min_norm'
    , 'price_max_norm'
    
    
    ,'next_item_cnt', 
    'next_item_normalized_weight',
    'discount_weight', 'following_weight', 
    'following_discount_weight', 'previous_weight', 
    'previous_discount_weight',
    'item_cnt', 
    'pos_weighted_cnt',
    'avg_pos', 'median_pos'
    
     ,'whether_w2v','whether_nic', 'whether_nfi', 'whether_covisit',
    'strategy_num',
    'vec_0', 'vec_1', 'vec_2', 'vec_3', 'vec_4', 'vec_5', 'vec_6', 'vec_7', 'vec_8', 'vec_9', 'vec_10', 'vec_11', 'vec_12', 'vec_13', 'vec_14', 'vec_15', 'vec_16', 'vec_17', 'vec_18', 'vec_19', 'vec_20', 'vec_21', 'vec_22',
    'vec_23', 'vec_24', 'vec_25', 'vec_26', 'vec_27', 
    'vec_28', 'vec_29', 'vec_30', 'vec_31',
    'nicnic_weight'
]

categorical_feature=['locale', 
                     'whether_w2v','whether_nic', 'whether_nfi', 'whether_covisit'
                    ]

# Function

## More Features 

In [13]:
session_features = pl.scan_parquet('../data/session_item_features/session_features.parquet')
item_features = pl.scan_parquet('../data/session_item_features/item_features.parquet')
product_pl = (pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 
                                          'products_train.parquet'))
                  # .with_columns(
                  #     pl.when(pl.col('locale')=='DE').then(1).when(pl.col('locale')=='DE')
                  #       .then(2)
                  #       .otherwise(3).alias('locale')
                  # )
             )

def get_additional_feature(data_pl, session_features=session_features, product_pl=product_pl, item_features=item_features):
    session_join_keys = ['session_id']
    session_fea_cols = ['mean_price', 'last_price', 'min_price', 'max_price']
    data_pl = (
        data_pl
         .with_columns(
            pl.col('session_id').cast(pl.Int64)
        )
        .join(session_features.select(session_join_keys+session_fea_cols+['locale']),
                                    how='left', on=session_join_keys, suffix='_str')
            .join(item_features, how='left',
                  left_on='next_item_prediction',
                  right_on='prev_items')
            .join(product_pl.select(['id', 'locale', 'price']), how='left', right_on=['id', 'locale']
                 , left_on=['next_item_prediction', 'locale_str'])
            .with_columns(
              pl.col('price').truediv(pl.col('mean_price')).alias('price_mean_norm')
              , pl.col('price').truediv(pl.col('last_price')).alias('price_last_norm')
              , pl.col('price').truediv(pl.col('min_price')).alias('price_min_norm')
              , pl.col('price').truediv(pl.col('max_price')).alias('price_max_norm')
              # , pl.when(pl.col('min_price')==pl.col('max_price')).then(1).otherwise((pl.col('price')-pl.col('min_price'))/(pl.col('min_price')-pl.col('max_price'))).alias('price_norm')
            )
            .select(
                pl.all().exclude(session_fea_cols+['locale_str', 'price'])
            )

    )
    return data_pl

# Read data

In [14]:

eval_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))

# df_sess.head(3).collect()
test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, f'sessions_test_{task}.parquet'), n_rows=SAMPLE_NUM).with_columns(pl.col('prev_items').apply(str2list))
test4task3_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task3.parquet'), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))



In [15]:
! ls {ranker_train_data_dir}

eval.parquet  test.parquet  test4task3.parquet	train


In [16]:
ranker_train_data_dir

'../data/rank_train_data_task2_v3'

In [17]:
train_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'train/' '*.parquet'), n_rows=SAMPLE_NUM)
# train_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'train.parquet'), n_rows=SAMPLE_NUM)


eval_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet'), n_rows=SAMPLE_NUM)


In [18]:
test_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'test.parquet'), n_rows=SAMPLE_NUM)
test4task3_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'test4task3.parquet'), n_rows=SAMPLE_NUM)

In [19]:
# train_candidates.collect().shape

In [20]:
# ! ls {ranker_train_data_dir}/train

In [21]:
dic = eval_candidates.schema

In [22]:
dic.keys()

dict_keys(['session_id', 'next_item_prediction', 'w2v_weight', 'whether_w2v', 'whether_nic', 'whether_nicnic', 'whether_nfi', 'whether_covisit', 'strategy_num', 'nicnic_weight', 'next_item', 'locale', 'prev_length', 'rec_num', 'target', 'next_item_weight', 'next_item_cnt', 'next_item_weight_nfi', 'next_item_normalized_weight', 'next_item_weight_co_visit', 'discount_weight', 'following_weight', 'following_discount_weight', 'previous_weight', 'previous_discount_weight', 'last_item_similarity', 'vec_0', 'vec_1', 'vec_2', 'vec_3', 'vec_4', 'vec_5', 'vec_6', 'vec_7', 'vec_8', 'vec_9', 'vec_10', 'vec_11', 'vec_12', 'vec_13', 'vec_14', 'vec_15', 'vec_16', 'vec_17', 'vec_18', 'vec_19', 'vec_20', 'vec_21', 'vec_22', 'vec_23', 'vec_24', 'vec_25', 'vec_26', 'vec_27', 'vec_28', 'vec_29', 'vec_30', 'vec_31'])

In [23]:
train_candidates.schema

{'session_id': Float64,
 'next_item_prediction': Utf8,
 'w2v_weight': Float64,
 'whether_w2v': Int64,
 'whether_nic': Int64,
 'whether_nicnic': Int64,
 'whether_nfi': Int64,
 'whether_covisit': Int64,
 'strategy_num': Int64,
 'nicnic_weight': Float64,
 'next_item': Utf8,
 'locale': Float64,
 'prev_length': Float64,
 'rec_num': Float64,
 'target': Float64,
 'next_item_weight': Float64,
 'next_item_cnt': Float64,
 'next_item_weight_nfi': Float32,
 'next_item_normalized_weight': Float64,
 'next_item_weight_co_visit': Float32,
 'discount_weight': Float32,
 'following_weight': Float32,
 'following_discount_weight': Float32,
 'previous_weight': Float32,
 'previous_discount_weight': Float32,
 'last_item_similarity': Float32,
 'vec_0': Float64,
 'vec_1': Float64,
 'vec_2': Float64,
 'vec_3': Float64,
 'vec_4': Float64,
 'vec_5': Float64,
 'vec_6': Float64,
 'vec_7': Float64,
 'vec_8': Float64,
 'vec_9': Float64,
 'vec_10': Float64,
 'vec_11': Float64,
 'vec_12': Float64,
 'vec_13': Float64,
 '

# Ranker 

## Train model

In [24]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="gbdt",
    n_estimators=estimator, 
    importance_type='gain',
    eval_at=[3]
)

### eval data for training 

In [25]:
# eval_candidates.head().collect()

In [26]:
# train_candidates.collect().shape

In [27]:
# test_candidates.head(3).collect()

In [28]:
# train_data_source

In [29]:
train_data_source = 'train' # train or 'eval'

if train_data_source == 'eval':
    target_df = eval_candidates
    negative_sampling = 0.1

    cols = ['session_id', 'next_item', 'next_item_prediction', 'target']
    sample_pl = target_df.select(cols)
    pos = sample_pl.filter(pl.col('target')==1)
    neg = (sample_pl
               .filter(pl.col('target')==0)
          )
    neg_original_len = neg.select('session_id').collect().shape[0]
    print(f'Original negative num: {neg_original_len}')
    neg = neg.groupby('session_id').agg(
        pl.all().sample(frac=negative_sampling)
        )
    neg = neg.explode(neg.columns[1:])
    # # print(neg.shape)
    # # print(neg.head().collect())
    sample_pl = pl.concat([pos, neg], how='vertical')
    target_df = sample_pl.join(target_df, how='left', on=cols).collect().sort('session_id').lazy()
    del sample_pl
else:
    target_df = train_candidates
    del train_candidates

### Train 

In [30]:
# target_df.schema

In [31]:
# target_train_candidates.head().collect()

In [32]:
# session_features.head().collect()

In [33]:
# product_pl.head().collect()

In [34]:
# product_pl.head().collect()

In [35]:
# session_features.schema

In [36]:
# target_train_candidates

In [37]:
# data_pl = target_train_candidates

In [38]:
# data_pl.select('locale').collect()

In [39]:
# data_pl.schema

In [40]:
# session_features.schema

In [41]:
# session_join_keys = ['session_id']
# session_fea_cols = ['mean_price', 'last_price', 'min_price', 'max_price']
# # data_pl = 
# (
#     data_pl
#     .with_columns(
#         pl.col('session_id').cast(pl.Int64)
#     )
#     .join(session_features.select(session_join_keys+session_fea_cols+['locale']),
#                                 how='left', on=session_join_keys, suffix='_str')
#         .join(product_pl.select(['id', 'locale', 'price']), how='left', right_on=['id', 'locale']
#              , left_on=['next_item_prediction', 'locale_str'])
#         .with_columns(
#           pl.col('price').truediv(pl.col('mean_price')).alias('price_mean_norm')
#           , pl.col('price').truediv(pl.col('last_price')).alias('price_last_norm')
#           , pl.col('price').truediv(pl.col('min_price')).alias('price_min_norm')
#           , pl.col('price').truediv(pl.col('max_price')).alias('price_max_norm')
#           # , pl.when(pl.col('min_price')==pl.col('max_price')).then(1).otherwise((pl.col('price')-pl.col('min_price'))/(pl.col('min_price')-pl.col('max_price'))).alias('price_norm')
#         )
#         .select(
#             pl.all().exclude(session_fea_cols+['locale_str', 'price'])
#         )

# ).head().collect()#.select(['session_id','next_item_prediction', 'locale_str'])

In [64]:
target_train_candidates = target_df
# del target_df

In [65]:
target_train_candidates = get_additional_feature(data_pl=target_train_candidates)

In [66]:
# target_train_candidates.collect()

In [67]:

pos = target_train_candidates.filter(pl.col('target')==1)
neg = (target_train_candidates
           .filter(pl.col('target')==0)
      )
neg = neg.groupby('session_id').agg(
    pl.all().sample(frac=0.3)
    )
neg = neg.explode(neg.columns[1:])
target_train_candidates = pl.concat([pos, neg], how='vertical')


In [68]:
train_session_id = target_train_candidates.select(pl.col('session_id')).unique().collect()

session_num = train_session_id.shape[0]

In [69]:
random_pl = train_session_id.with_columns(
        pl.Series(name='random', values=np.random.uniform(size=session_num))
)

In [70]:
train_candidates = target_train_candidates.join(random_pl.lazy(), how='left', on='session_id')

sampled_train_candidates = train_candidates.filter(pl.col('random')>eval_frac).sort('session_id', descending=False).collect()
sampled_eval_candidates = train_candidates.filter(pl.col('random')<=eval_frac).sort('session_id', descending=False).collect()

In [71]:
print('Train')
print(sampled_train_candidates.shape)
print(sampled_train_candidates.select('target').mean())
print()
print()
print('Eval')
print(sampled_eval_candidates.shape)
print(sampled_eval_candidates.select('target').mean())

Train
(4604291, 67)
shape: (1, 1)
┌─────────┐
│ target  │
│ ---     │
│ f64     │
╞═════════╡
│ 0.02106 │
└─────────┘


Eval
(502804, 67)
shape: (1, 1)
┌──────────┐
│ target   │
│ ---      │
│ f64      │
╞══════════╡
│ 0.021267 │
└──────────┘


In [72]:
# sampled_train_candidates.head()

<!-- no positive items ->  -->

In [73]:
# sampled_train_candidates.filter(pl.col('session_id')==0).sort('target', descending=True)

In [74]:
# sampled_train_candidates.filter(pl.col('session_id')==5).sort('target', descending=True)

In [75]:
# sampled_train_candidates.head()

In [76]:
# sampled_train_candidates.groupby('session_id').agg(pl.count())

In [77]:
session_lengths_train = sampled_train_candidates.groupby('session_id').agg(pl.count()).select('count').to_pandas()['count'].values.tolist()
session_lengths_eval = sampled_eval_candidates.groupby('session_id').agg(pl.count()).select('count').to_pandas()['count'].values.tolist()

In [78]:
pd.DataFrame(session_lengths_train).describe([0.05*i for i in range(20)])

,0
count,96958.000000
mean,47.487479
std,22.423329
min,1.000000
0%,1.000000
5%,30.000000
10%,31.000000
15%,32.000000
20%,33.000000
25%,34.000000


In [79]:
pd.DataFrame(session_lengths_eval).describe([0.05*i for i in range(20)])

,0
count,10692.000000
mean,47.026188
std,21.824378
min,1.000000
0%,1.000000
5%,30.000000
10%,31.000000
15%,32.000000
20%,33.000000
25%,34.000000


In [80]:
ranker = ranker.fit(
    X=sampled_train_candidates[feature_cols].to_pandas(),
    y=sampled_train_candidates[target].to_pandas(),
    group=session_lengths_train,
    eval_set=[(sampled_train_candidates[feature_cols].to_pandas(), sampled_train_candidates[target].to_pandas()),
           (sampled_eval_candidates[feature_cols].to_pandas(), sampled_eval_candidates[target].to_pandas())
             ],
    eval_group=[session_lengths_train,
                session_lengths_eval
               ]
    , categorical_feature=categorical_feature
    , early_stopping_rounds=early_stop_rounds
    # , callbacks=[early_stopping(early_stop_rounds)]
)

/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/sklearn.py:621: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib

[1]	valid_0's ndcg@3: 0.721552	valid_1's ndcg@3: 0.725081
[2]	valid_0's ndcg@3: 0.685949	valid_1's ndcg@3: 0.691427
[3]	valid_0's ndcg@3: 0.679884	valid_1's ndcg@3: 0.685373
[4]	valid_0's ndcg@3: 0.678159	valid_1's ndcg@3: 0.682864
[5]	valid_0's ndcg@3: 0.677448	valid_1's ndcg@3: 0.681886
[6]	valid_0's ndcg@3: 0.677109	valid_1's ndcg@3: 0.681872
[7]	valid_0's ndcg@3: 0.676974	valid_1's ndcg@3: 0.681713
[8]	valid_0's ndcg@3: 0.676783	valid_1's ndcg@3: 0.680897
[9]	valid_0's ndcg@3: 0.677042	valid_1's ndcg@3: 0.680915
[10]	valid_0's ndcg@3: 0.676873	valid_1's ndcg@3: 0.679396
[11]	valid_0's ndcg@3: 0.676642	valid_1's ndcg@3: 0.680175


In [81]:
# print(sampled_train_candidates.estimated_size(unit='gb'))
# del sampled_train_candidates

In [60]:
# print(sampled_eval_candidates.estimated_size(unit='gb'))
# del sampled_eval_candidates

## Save model 

In [61]:
debug

False

In [278]:
rank_model_dir

'../model_training/task2_rank_lgbm_v3'

In [279]:
if not debug:
    joblib.dump(
            value=ranker,
            filename=os.path.join(rank_model_dir, 'model.pkl')
    )

In [280]:
if not debug:
    del ranker

## Load Model 

In [281]:
debug

False

In [282]:
if not debug:
    ranker = joblib.load(os.path.join(rank_model_dir, 'model.pkl'))
ranker

LGBMRanker(eval_at=[3], importance_type='gain', metric='ndcg',
           n_estimators=1000, objective='lambdarank')

### Importance 

In [82]:
impotant_df = pd.DataFrame(
    {
        'features': ranker.feature_name_,
        'importance': ranker.feature_importances_
    }
).sort_values('importance', ascending=False)
impotant_df

,features,importance
12,next_item_normalized_weight,418416.239868
10,price_max_norm,86756.703125
59,nicnic_weight,32601.771118
7,price_mean_norm,24634.124023
5,last_item_similarity,19446.424194
17,previous_discount_weight,13169.387390
19,pos_weighted_cnt,1917.819946
4,w2v_weight,1850.819946
13,discount_weight,1011.890015
26,strategy_num,632.869019


# Eval ranker 

In [290]:
def rank_rec(candidate_pl, feature_cols, ranker, topn=100):

    candidate_pl = candidate_pl.collect()
    print(f"candidate pl shape: {candidate_pl.shape}")
    inference = ranker.predict(candidate_pl[feature_cols].to_pandas())
    test_result = (candidate_pl
         .lazy()
         .with_columns(
             pl.Series(name='predict', values=inference)
         )
         .with_columns(
            pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
         )
         .sort(['session_id', 'rank'])
         .filter(pl.col('rank')<=topn)
         .groupby(['session_id'])
         .agg(
             pl.col('next_item_prediction')
             , pl.col('next_item').unique()#.arr.get(0).cast(pl.Utf8)
         ).with_columns(
             pl.col('next_item').arr.get(0).alias('next_item')
         )
    )
    return test_result


In [291]:
# target_train_candidates.head().collect()

In [286]:
eval_candidates = get_additional_feature(data_pl=eval_candidates)

In [287]:
ranker_eval_pl = rank_rec(candidate_pl=eval_candidates,
                         feature_cols=feature_cols
                         , ranker=ranker
                        # , topn=10
                         )

candidate pl shape: (4998123, 66)


In [288]:
model_eval(target_df=eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id'))


total_sessions,mrr,recall@20,recall@100
u32,f64,f64,f64
33333,0.376,0.6317,0.7114


In [292]:
# model_eval(target_df=eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id'))


In [293]:
# model_eval(target_df=eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id'))


In [294]:
# model_eval(target_df=eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id'))


In [295]:
# model_eval(target_df=eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id'))


In [296]:
# model_eval(target_df=eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id'))

In [297]:
# model_eval(target_df=eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id'))

# Test4task3 

In [298]:
# only 1 is needed
topn=1

test4task3_candidates = get_additional_feature(data_pl=test4task3_candidates)
test4task3_candidates = test4task3_candidates.collect()
inference = ranker.predict(test4task3_candidates[feature_cols].to_pandas())
test4task3_result = (test4task3_candidates
     .lazy()
     .with_columns(
         pl.Series(name='predict', values=inference)
     )
     .with_columns(
        pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
     )
     .sort(['session_id', 'rank'])
     .filter(pl.col('rank')<=topn)
     .groupby(['session_id'])
     .agg(
         pl.col('next_item_prediction')
     )
)
# test4task3_result.head().collect()
predictions = test4task3_pl.join(test4task3_result, how='left', on='session_id').collect()
# [['locale', 'next_item_prediction']].to_pandas()

In [299]:
test4task3_sub_file = f'../data/sub_files/{task}_test4task3_{rank_model_version}.parquet'
print(test4task3_sub_file)
predictions.write_parquet(test4task3_sub_file)
predictions.select('locale').to_series().value_counts()

../data/sub_files/task2_test4task3_task2_rank_lgbm_v3.parquet


locale,counts
str,u32
"""ES""",6422
"""IT""",10000
"""FR""",10000


# fallback logics 

In [300]:
# pop_model = pl.scan_parquet('../model_training/popular_v1/popular_rec.parquet')

In [301]:
# pop_model.collect().shape

# Test inference 

In [302]:
test_candidates = get_additional_feature(data_pl=test_candidates)
test_candidates = test_candidates.collect()
inference = ranker.predict(test_candidates[feature_cols].to_pandas())
test_result = (test_candidates
     .lazy()
     .with_columns(
         pl.Series(name='predict', values=inference)
     )
     .with_columns(
        pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
     )
     .sort(['session_id', 'rank'])
     .filter(pl.col('rank')<=100)
     .groupby(['session_id'])
     .agg(
         pl.col('next_item_prediction')
     )
)
# test_result.head().collect()
predictions = test_pl.join(test_result, how='left', on='session_id').collect()[['locale', 'next_item_prediction']].to_pandas()

# data validation 

In [303]:
# test_result.head().collect()

In [304]:
predictions_pl = (
    test_pl.join(test_result, how='left', on='session_id')
        .with_columns(
            pl.when(pl.col('next_item_prediction').is_null())
                .then(pl.col('prev_items'))
                .otherwise(pl.col('next_item_prediction')).alias('next_item_prediction')
        )
).collect()

In [305]:
# predictions_pl.head()

In [306]:
predictions = (predictions_pl[['locale', 'next_item_prediction']].to_pandas())

In [307]:
# predictions

In [308]:
prediction_summary = (
    predictions_pl
        .explode('next_item_prediction')
        .with_columns(
            pl.col('next_item_prediction').is_null().alias('whether_null')
        )
        .groupby('session_id')
        .agg(
            pl.col('whether_null').sum().alias('null_num')
            , pl.count().alias('rec_num')
        )
)

In [309]:
# prediction_summary.select('rec_num').to_pandas().describe([0.01*i for i in range(15)])

In [310]:
# predictions_plfilter(pl.col('session_id')==4338464).collect()

In [311]:
# test_pl.filter(pl.col('session_id')==4338464).collect()

In [312]:
# test_result.filter(pl.col('session_id')==4338464).collect()

In [313]:
assert (
    prediction_summary.filter(pl.col('null_num')>0)
).shape[0] == 0

In [314]:
check_predictions(predictions, test_sessions=test_pl.collect().to_pandas(), 
                  # check_products=True, product_df=products
                 )
# Its important that the parquet file you submit is saved with pyarrow backend

In [315]:
assert (
    pl.from_pandas(predictions)
        .explode('next_item_prediction')
        .filter(
            pl.col('next_item_prediction').is_null()
                     )
).shape[0] == 0

In [316]:
assert predictions.shape[0] == test_pl.collect().shape[0]

In [317]:
sub_file

'../data/sub_files/task2_rank_lgbm_v3.parque'

In [318]:
if not debug:
    predictions.to_parquet(sub_file, engine='pyarrow')

In [319]:
! ls -al {sub_file}

-rw-r--r-- 1 jupyter jupyter 28672972 Jun  8 21:57 ../data/sub_files/task2_rank_lgbm_v3.parque


In [320]:
# sampled_train_candidates.sample(20)

# Submit result 

In [187]:
# # You can submit with aicrowd-cli, or upload manually on the challenge page.
# !aicrowd submission create -c task-2-next-product-recommendation-for-underrepresented-languages -f {sub_file}

# debug 

In [394]:
sub_pl = pl.scan_parquet(sub_file)

In [395]:
sub_pl.head().collect()

locale,next_item_prediction
str,list[str]
"""ES""","[""B074RZKWMD"", ""B074574XPT"", … ""B09P15NQ3D""]"
"""ES""","[""B09M8LNB61"", ""B09J4T4JF5"", … ""B08P5C768V""]"
"""ES""","[""B0B461V1RB"", ""B0B4614XL6"", … ""B09TL4RXK4""]"
"""ES""","[""B07R4VG9X3"", ""B07R4WKSSV"", … ""B09VQD8V9T""]"
"""ES""","[""B09G9LF91K"", ""B09G97SLGS"", … ""B09ZD7D5LB""]"


In [396]:
null_prediciton = (
    sub_pl.explode('next_item_prediction')
        .filter(
            pl.col('next_item_prediction').is_null()
        )
).collect()

In [398]:
null_prediciton

locale,next_item_prediction
str,str
"""ES""",null
"""ES""",null
"""ES""",null
"""ES""",null
"""ES""",null
"""ES""",null
"""ES""",null
"""ES""",null
"""ES""",null


#  Case Study from taks2

In [92]:
product_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 
                                          'products_train.parquet'))

In [93]:
model_eval(target_df=eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id'))

total_sessions,mrr,recall@20,recall@100
u32,f64,f64,f64
33333,0.3242,0.5594,0.6419


In [94]:
target_df = eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id')

In [131]:
# target_df.head().collect()

In [96]:
eval_final = (
        target_df
        .lazy()
        .with_columns(
            pl.col('next_item_prediction').cast(pl.List(pl.Utf8))
        )
        .with_columns(
            pl.concat_list([pl.col('next_item'), pl.col('next_item_prediction')]).alias('mrr')
        )
        .with_columns(
            pl.col('mrr').arr.eval(
                pl.arg_where(pl.element()==pl.element().first())
            )
        ).with_columns(
            pl.col('mrr').arr.eval(
                pl.when(pl.element()==0).then(0).otherwise(1/pl.element())
            )
        ).with_columns(
            pl.col('mrr').arr.sum()
            , pl.col('next_item_prediction').arr.head(1).arr.contains(pl.col('next_item')).alias('recall@1')
            , pl.col('next_item_prediction').arr.head(3).arr.contains(pl.col('next_item')).alias('recall@3')
            , pl.col('next_item_prediction').arr.head(20).arr.contains(pl.col('next_item')).alias('recall@20')
            , pl.col('next_item_prediction').arr.head(100).arr.contains(pl.col('next_item')).alias('recall@100')

        )
)
final_res = eval_final.select(
        pl.count().alias('total_sessions')
        , pl.col('mrr').mean().round(4)
        , pl.col('recall@1').mean().round(4)
        , pl.col('recall@3').mean().round(4)
        , pl.col('recall@20').mean().round(4)
        , pl.col('recall@100').mean().round(4)

    ).collect()
final_res

total_sessions,mrr,recall@1,recall@3,recall@20,recall@100
u32,f64,f64,f64,f64,f64
33333,0.3242,0.237,0.3675,0.5594,0.6419


In [98]:
# final_res

In [99]:
# eval_final.filter(pl.col('recall@1')==True).head().collect()

In [100]:
# eval_final.head().collect()

## recall@3 = True and recall@1=False 

In [101]:
all_cases = eval_final.filter(pl.col('recall@1')==False).filter(pl.col('recall@3')==True)

In [102]:
all_cases = (
    all_cases.with_columns(
        pl.col('next_item_prediction').arr.head(3).alias('next_item_prediction')
    )
)

In [150]:
all_cases.collect().sample(5)

prev_items,next_item,locale,session_id,next_item_prediction,next_item_right,mrr,recall@1,recall@3,recall@20,recall@100
list[str],str,str,i64,list[str],str,f64,bool,bool,bool,bool
"[""B00NPXEEC4"", ""B07JN5C1S3"", … ""B0B3JS3NFN""]","""B09NQHCKJK""","""IT""",3575764,"[""B08P4YPB8Q"", ""B0092QK8N2"", ""B09NQHCKJK""]","""B09NQHCKJK""",0.333333,false,true,true,true
"[""B06Y2H5JGX"", ""B0811QN9PN""]","""B0B1J56P98""","""FR""",3407839,"[""B09ZXWHPRD"", ""B0B1J56P98"", ""B07GX1WYFC""]","""B0B1J56P98""",0.5,false,true,true,true
"[""B08BYJ9Q65"", ""B08HRG6W11"", ""B08BYJ9Q65""]","""B08TLYM129""","""IT""",3537237,"[""B09G5N3D36"", ""B08TLYM129"", ""B086XFFD5Y""]","""B08TLYM129""",0.5,false,true,true,true
"[""B01N1NR31G"", ""B01MZ5ELRT""]","""B01F549LFK""","""IT""",3538047,"[""B01N5L1J0P"", ""B01F549LFK"", ""B01MSUK566""]","""B01F549LFK""",0.5,false,true,true,true
"[""B003Z0HU0K"", ""B003Z0HU0K"", ""B09NFV8PBV""]","""B09NGFB7KD""","""FR""",3367530,"[""B09NFT49PW"", ""B09NGFB7KD"", ""2226328432""]","""B09NGFB7KD""",0.5,false,true,true,true


## Function 

In [143]:
# def print_single_case(session_id, all_cases):
#     feature = (
#      all_cases
#         .select(['next_item_prediction'])
#         .filter(pl.col('session_id')==session_id).explode('next_item_prediction')
#         .join(
#             eval_candidates.filter(pl.col('session_id')==session_id)
#             , how='left'
#             , on='next_item_prediction'
#         )
# )  
#     print()
#     print("features")
#     print(feature.collect())
#     print()
#     print()
#                   # e
    

### 3272776 --> hard to locate problem; `next_item_weight_co_visit` 

In [132]:
session_id = 3272776

In [142]:
# print_single_case(session_id=session_id, all_cases=all_cases)

In [133]:
single_case = all_cases.filter(pl.col('session_id')==session_id).collect()
single_case

prev_items,next_item,locale,session_id,next_item_prediction,next_item_right,mrr,recall@1,recall@3,recall@20,recall@100
list[str],str,str,i64,list[str],str,f64,bool,bool,bool,bool
"[""B0B2K87MXS"", ""B0B2KBH114"", … ""B0B2K87MXS""]","""B0B4612MTM""","""ES""",3272776,"[""B09PNDJJZ9"", ""B08FMPXDTJ"", ""B0B4612MTM""]","""B0B4612MTM""",0.333333,false,true,true,true


In [134]:
show_single_case(session_id=session_id, session_pl=all_cases, product_pl=product_pl)

++++++++++++++++++++
0: B0B2K87MXS
++++++++++++++++++++
===title====
    XIAOMI Xiaomi Smartphone redmi 9c NFC 3/64gb Verde Libre

===desc====
    Alta calidad

unique_id                               1416732
price                                    116.76
brand                                    Xiaomi
color                              Aurora Green
size         Redmi 9C NFC 3GB+64GB Aurora Green
model                     REDMI 9C NFC 3-64 GRE
material                                   None
author                                     None
Name: 0, dtype: object


++++++++++++++++++++
1: B0B2K87MXS
++++++++++++++++++++
===title====
    XIAOMI Xiaomi Smartphone redmi 9c NFC 3/64gb Verde Libre

===desc====
    Alta calidad

unique_id                               1416732
price                                    116.76
brand                                    Xiaomi
color                              Aurora Green
size         Redmi 9C NFC 3GB+64GB Aurora Green
model                     RED

In [135]:
show_single_case(session_id=session_id, session_pl=all_cases, product_pl=product_pl
                , item_list_col='next_item_prediction')

++++++++++++++++++++
0: B08FMPXDTJ
++++++++++++++++++++
===title====
    Xiaomi Redmi 9C NFC-Smartphone de 6.53" HD+ (3GB+64GB, 3x cámara trasera de 13MP con IA, MediaTek Helio G35, Batería 5000 mAh, 10 W carga rápida), Gris

===desc====
    Batería de alta capacidad de 5000 mah con carga rápida de 10w.

unique_id          1429975
price                121.0
brand               Xiaomi
color        Midnight Gray
size            3GB + 64GB
model        6941059651307
material              None
author              Xiaomi
Name: 0, dtype: object


++++++++++++++++++++
1: B09PNDJJZ9
++++++++++++++++++++
===title====
    Xiaomi Redmi 9C - Smartphone 128GB, 4GB RAM, Dual Sim, Aurora Green

===desc====
    Batería: 5000 mAh

unique_id                            1430860
price                                  142.0
brand                                 Xiaomi
color                           Aurora Green
size         Redmi 9C 4GB+128GB Aurora Green
model                              MZB0A46EU
materi

In [136]:
feature = (
     all_cases
        .select(['next_item_prediction'])
        .filter(pl.col('session_id')==session_id).explode('next_item_prediction')
        .join(
            eval_candidates.filter(pl.col('session_id')==session_id)
            , how='left'
            , on='next_item_prediction'
        )
)  

feature.collect()
                  # e

next_item_prediction,session_id,w2v_weight,next_item,locale,prev_length,target,next_item_weight,next_item_weight_nfi,next_item_weight_co_visit,last_item_similarity,price_mean_norm,price_last_norm,price_min_norm,price_max_norm
str,i64,f64,str,i32,u32,i32,f64,f32,f32,f32,f64,f64,f64,f64
"""B09PNDJJZ9""",3272776,0.97,"""B0B4612MTM""",3,4,0,1.01,5.83,12.0,0.992541,1.321176,1.21617,1.449127,1.21617
"""B08FMPXDTJ""",3272776,0.995,"""B0B4612MTM""",3,4,0,0.26,2.25,8.0,0.994898,1.125791,1.036314,1.23482,1.036314
"""B0B4612MTM""",3272776,0.81,"""B0B4612MTM""",3,4,1,0.26,3.95,11.0,0.97156,1.292799,1.190048,1.418002,1.190048


### 3373971 -> reply on NIC too much; `w2v_weight` -> not zero for ground true 

In [145]:
session_id = 3373971

In [146]:
feature = (
     all_cases
        .select(['next_item_prediction'])
        .filter(pl.col('session_id')==session_id).explode('next_item_prediction')
        .join(
            eval_candidates.filter(pl.col('session_id')==session_id)
            , how='left'
            , on='next_item_prediction'
        )
)  

feature.collect()
                  # e

next_item_prediction,session_id,w2v_weight,next_item,locale,prev_length,target,next_item_weight,next_item_weight_nfi,next_item_weight_co_visit,last_item_similarity,price_mean_norm,price_last_norm,price_min_norm,price_max_norm
str,i64,f64,str,i32,u32,i32,f64,f32,f32,f32,f64,f64,f64,f64
"""B015TJ8NM8""",3373971,0.0,"""B000GIPJ02""",3,5,0,0.76,5.16,15.0,0.959601,0.772395,1.0,1.0,0.564819
"""B000GIPJ02""",3373971,0.89,"""B000GIPJ02""",3,5,1,0.51,3.0,5.0,0.853157,1.315306,1.702893,1.702893,0.961826
"""B002DYIZPS""",3373971,0.0,"""B000GIPJ02""",3,5,0,0.0,0.5,1.0,0.793911,1.367509,1.770479,1.770479,1.0


In [147]:
show_single_case(session_id=session_id, session_pl=all_cases, product_pl=product_pl)

++++++++++++++++++++
0: B000GIPJ0M
++++++++++++++++++++
===title====
    Optimum Nutrition Serious Mass, Mass Gainer avec Whey, Proteines Musculation Prise de Masse avec Vitamines, Creatine et Glutamine, Chocolat, 16 Portions, 5.45kg, l'Emballage Peut Varier

===desc====
    AUGMENTATEUR DE MASSE MUSCULAIRE avec un nombre exorbitant de 1250 calories et 50 g de whey protéines par portion. Contient moins de 5 g de graisse !

unique_id               1469992
price                     62.99
brand         Optimum Nutrition
color                  Chocolat
size         5.45 kg (Lot de 1)
model                   1101529
material              Palstique
author                     None
Name: 0, dtype: object


++++++++++++++++++++
1: B09RMWFZ8W
++++++++++++++++++++
===title====
    Optimum Nutrition | Serious Mass (2,73kg) | Gainers | Puissant accélérateur de masse

===desc====
    Augmentation de la prise de masse

unique_id               1470603
price                     36.99
brand         Opti

In [148]:
show_single_case(session_id=session_id, session_pl=all_cases, product_pl=product_pl
                , item_list_col='next_item_prediction')

++++++++++++++++++++
0: B015TJ8NM8
++++++++++++++++++++
===title====
    Optimum Nutrition Serious Mass, Mass Gainer avec Whey, Proteines Musculation Prise de Masse avec Vitamines, Creatine et Glutamine, Chocolat Beurre de Cacahuète, 8 Portions, l'Emballage Peut Varier

===desc====
    AUGMENTATEUR DE MASSE MUSCULAIRE avec un nombre incroyable de 1250 calories et 50 g de whey protéines par portion. Contient moins de 5 g de graisse

unique_id                          1459015
price                                36.99
brand        Glanbia Performance Nutrition
color         Chocolat Beurre de Cacahuète
size                    2.73 kg (Lot de 1)
model                              1101541
material                              None
author                                None
Name: 0, dtype: object


++++++++++++++++++++
1: B000GIPJ02
++++++++++++++++++++
===title====
    Optimum Nutrition Serious Mass, Mass Gainer avec Whey, Proteines Musculation Prise de Masse avec Vitamines, Creatine et Gl

### 3407839 w2v_weight is 0??;  看不出来？？？

In [152]:
session_id = 3407839

In [153]:
feature = (
     all_cases
        .select(['next_item_prediction'])
        .filter(pl.col('session_id')==session_id).explode('next_item_prediction')
        .join(
            eval_candidates.filter(pl.col('session_id')==session_id)
            , how='left'
            , on='next_item_prediction'
        )
)  

feature.collect()
                  # e

next_item_prediction,session_id,w2v_weight,next_item,locale,prev_length,target,next_item_weight,next_item_weight_nfi,next_item_weight_co_visit,last_item_similarity,price_mean_norm,price_last_norm,price_min_norm,price_max_norm
str,i64,f64,str,i32,u32,i32,f64,f32,f32,f32,f64,f64,f64,f64
"""B09ZXWHPRD""",3407839,0.0,"""B0B1J56P98""",3,2,0,1.01,1.0,1.0,0.951582,1.258554,1.054069,1.561474,1.054069
"""B0B1J56P98""",3407839,0.0,"""B0B1J56P98""",3,2,1,1.01,1.0,1.0,0.943084,1.290833,1.081103,1.601522,1.081103
"""B07GX1WYFC""",3407839,0.96,"""B0B1J56P98""",3,2,0,0.0,0.0,0.0,0.963218,0.659458,0.552311,0.818182,0.552311


In [154]:
show_single_case(session_id=session_id, session_pl=all_cases, product_pl=product_pl)

++++++++++++++++++++
0: B06Y2H5JGX
++++++++++++++++++++
===title====
    ANSIO Fer à repasser à vapeur en céramique, 2000 W, cordon d'alimentation 3m / 10ft, réservoir d'eau 200ml - violet/blanc

===desc====
    CONTRÔLE DE TEMPÉRATURE RÉGLABLE: Le réglage de température précis permet de faire correspondre le type de vêtement pour une finition parfaite.

unique_id      1489916
price            24.97
brand            ANSIO
color           Violet
size              None
model             None
material     Céramique
author            None
Name: 0, dtype: object


++++++++++++++++++++
1: B0811QN9PN
++++++++++++++++++++
===title====
    Yabano Fer a Repasser, Fer à Vapeur 2400W, 350 ml Réservoir, Fonction Pressing 320g/min, Semelle en Céramique, 2,5m Cable, Auto-nettoyage

===desc====
    💨2400 Watts, Semelle céramique à trous multiples.

unique_id                1491923
price                      36.99
brand                     Yabano
color                        Blu
size                   

In [155]:
show_single_case(session_id=session_id, session_pl=all_cases, product_pl=product_pl
                , item_list_col='next_item_prediction')


++++++++++++++++++++
0: B0B1J56P98
++++++++++++++++++++
===title====
    Cozeemax Fer à vapeur 2400 W, 320 g/min - Semelle en céramique - Nettoyage automatique et explosion - Arrêt de goutte - Réservoir d'eau de 300 ml - Vapeur verticale

===desc====
    Thermostat réglable et vapeur : un contrôle précis de la température et un contrôle de vapeur variable peuvent protéger vos vêtements dans les meilleures conditions. Parfait pour tous les tissus tels que le nylon, la soie, la laine, le coton, les jeans et le lin.

unique_id     1475018
price           39.99
brand        Cozeemax
color            None
size             None
model          AJ2016
material         None
author           None
Name: 0, dtype: object


++++++++++++++++++++
1: B09ZXWHPRD
++++++++++++++++++++
===title====
    Défroisseur vapeur vertical portable voyage, 280ml, 1500W Fer à vapeur pratique Chauffage rapide, vapeur 30g/min, pour vêtements, rideaux, meubles rembourrés

===desc====
    【Conception de sécurité】 Chaque

## why NIC so important

In [91]:
train_candidate_df = sampled_train_candidates.to_pandas()

In [93]:
train_candidate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1426023 entries, 0 to 1426022
Data columns (total 32 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   session_id                   1426023 non-null  int64  
 1   next_item                    1426023 non-null  object 
 2   next_item_prediction         1426023 non-null  object 
 3   target                       1426023 non-null  float64
 4   w2v_weight                   1426023 non-null  float64
 5   locale                       1426023 non-null  float64
 6   prev_length                  1426023 non-null  float64
 7   next_item_weight             1426023 non-null  float64
 8   next_item_cnt                1426023 non-null  float64
 9   next_item_weight_nfi         1426023 non-null  float32
 10  next_item_normalized_weight  1426023 non-null  float64
 11  next_item_weight_co_visit    1426023 non-null  float32
 12  discount_weight              1426023 non-n

In [106]:
# train_candidate_df.sample()

,session_id,next_item,next_item_prediction,target,w2v_weight,locale,prev_length,next_item_weight,next_item_cnt,next_item_weight_nfi,...,price_mean_norm,price_last_norm,price_min_norm,price_max_norm,random,item_cnt_right,pos_weighted_cnt_right,avg_pos_right,median_pos_right,random_right
810680,3462257,B08VDMK1V2,B07HGSN1TH,0.0,0.5,3.0,2.0,0.0,0.0,0.0,...,350877.193596,236686.390947,677966.102881,236686.390947,0.69289,13.0,6.144444,3.307692,3.0,0.72063


In [98]:
train_candidate_df[train_candidate_df['target']==1]['next_item_normalized_weight'].value_counts()/len(train_candidate_df[train_candidate_df['target']==1])

next_item_normalized_weight
1.000000    0.271494
0.000000    0.171137
0.500000    0.012475
0.666667    0.006478
0.250000    0.006436
              ...   
0.431915    0.000011
0.318996    0.000011
0.103903    0.000011
0.718095    0.000011
0.737723    0.000011
Name: count, Length: 26337, dtype: float64

In [96]:
train_candidate_df[train_candidate_df['target']==0]['next_item_normalized_weight'].value_counts()/len(train_candidate_df[train_candidate_df['target']==0])

next_item_normalized_weight
0.000000    7.097932e-01
1.000000    2.169385e-03
0.500000    9.897676e-04
0.250000    9.650047e-04
0.166667    7.744050e-04
                ...     
0.113402    7.503925e-07
0.003193    7.503925e-07
0.014369    7.503925e-07
0.008325    7.503925e-07
0.111607    7.503925e-07
Name: count, Length: 111764, dtype: float64

### session_id = 3272717

In [101]:
session_id = 3272717

In [102]:
train_candidate_df[train_candidate_df['session_id']==3272717]['next_item_normalized_weight']

0     0.178165
1     0.000000
2     0.000000
3     0.000000
4     0.000000
5     0.000000
6     0.000000
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.000000
12    0.000000
13    0.000000
Name: next_item_normalized_weight, dtype: float64

In [103]:
train_candidates.filter(pl.col('session_id')==session_id).collect()

session_id,next_item,next_item_prediction,target,w2v_weight,locale,prev_length,next_item_weight,next_item_cnt,next_item_weight_nfi,next_item_normalized_weight,next_item_weight_co_visit,discount_weight,following_weight,following_discount_weight,previous_weight,previous_discount_weight,last_item_similarity,item_cnt,pos_weighted_cnt,avg_pos,median_pos,price_mean_norm,price_last_norm,price_min_norm,price_max_norm,random,item_cnt_right,pos_weighted_cnt_right,avg_pos_right,median_pos_right,random_right
i64,str,str,f64,f64,f64,f64,f64,f64,f32,f64,f32,f32,f32,f32,f32,f32,f32,u32,f64,f64,f64,f64,f64,f64,f64,f64,u32,f64,f64,f64,f64
3272717,"""B07JFPYN5P""","""B07JFPYN5P""",1.0,1.0,3.0,2.0,0.136,17.0,24.429998,0.178165,46.0,31.710001,35.0,24.429998,11.0,7.28,0.977248,23,10.991667,2.956522,3.0,1.0,4.0,4.0,0.571429,0.995319,23,10.991667,2.956522,3.0,0.208178
3272717,"""B07JFPYN5P""","""B07B26DZZG""",0.0,0.15,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.741993,2,1.333333,2.0,2.0,1.1995,4.798,4.798,0.685429,0.995319,2,1.333333,2.0,2.0,0.208178
3272717,"""B07JFPYN5P""","""B08CB9TJXH""",0.0,0.65,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.780336,4,1.833333,3.25,3.0,null,null,null,null,0.995319,4,1.833333,3.25,3.0,0.208178
3272717,"""B07JFPYN5P""","""B0002IHFVM""",0.0,0.7,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.790319,12,6.533333,2.75,2.5,null,null,null,null,0.995319,12,6.533333,2.75,2.5,0.208178
3272717,"""B07JFPYN5P""","""B09BMXXW6N""",0.0,0.61,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.776615,60,32.755556,3.016667,2.0,4.2495,16.998,16.998,2.428286,0.995319,60,32.755556,3.016667,2.0,0.208178
3272717,"""B07JFPYN5P""","""B0B24TLGXR""",0.0,0.64,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.771123,311,171.51293,3.379421,2.0,0.4495,1.798,1.798,0.256857,0.995319,311,171.51293,3.379421,2.0,0.208178
3272717,"""B07JFPYN5P""","""B0B6CKW1RP""",0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,1.0,0.5,0.0,0.0,1.0,0.5,0.56156,23,16.177778,2.304348,1.0,2.159,8.636,8.636,1.233714,0.995319,23,16.177778,2.304348,1.0,0.208178
3272717,"""B07JFPYN5P""","""B06XNJ656V""",0.0,0.46,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.748781,15,8.485714,2.8,2.0,null,null,null,null,0.995319,15,8.485714,2.8,2.0,0.208178
3272717,"""B07JFPYN5P""","""B08VNLN7KK""",0.0,0.24,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.749546,94,47.158626,3.542553,2.0,0.4495,1.798,1.798,0.256857,0.995319,94,47.158626,3.542553,2.0,0.208178


In [105]:
! ls '../data/candidates/' | grep task2_v2

task2_eval_co_visit_task2_v2_True_top300.parquet
task2_eval_nfi_task2_v2_True_top100.parquet
task2_eval_nic_task2_v2_True_top100.parquet
task2_eval_w2v_task2_v2_True_top100.parquet
task2_test4task3_co_visit_task2_v2_True_top300.parquet
task2_test4task3_nfi_task2_v2_True_top100.parquet
task2_test4task3_nic_task2_v2_True_top100.parquet
task2_test4task3_w2v_task2_v2_True_top100.parquet
task2_test_co_visit_task2_v2_True_top300.parquet
task2_test_nfi_task2_v2_True_top100.parquet
task2_test_nic_task2_v2_True_top100.parquet
task2_test_w2v_task2_v2_True_top100.parquet
task2_train_co_visit_task2_v2_True_top300
task2_train_co_visit_task2_v2_True_top300.parquet
task2_train_nfi_task2_v2_True_top100
task2_train_nfi_task2_v2_True_top100.parquet
task2_train_nic_task2_v2_True_top100
task2_train_nic_task2_v2_True_top100.parquet
task2_train_w2v_task2_v2_True_top100
task2_train_w2v_task2_v2_True_top100.parquet


In [ ]:
# def re_process_data(task=task, 
#                        w2v_model_version=w2v_model_version, 
#                        nic_model_version=nic_model_version,
#                        nfi_model_version=nfi_model_version,
#                        covisit_model_version=covisit_model_version,
#                     model_for_eval=model_for_eval,
#                       w2v_topn=w2v_topn
#                       , nic_topn=nic_topn
#                        , nfi_topn=nfi_topn
#                        , covisit_topn=covisit_topn
#                     , train_parts=train_parts
#                       ):
    # parts = train_parts

data_type = 'train'
w2v_file = os.path.join(candidate_path, 
                       candidate_file_name.format(
                task=task
            , data_type=data_type
            , model_version=w2v_model_version
            , model_for_eval=model_for_eval
            , topn=w2v_topn
                       ))
nic_file = os.path.join(candidate_path,
            candidate_file_name.format(
                task=task
                , data_type=data_type
                , model_version=nic_model_version
                , model_for_eval=model_for_eval
                , topn=nic_topn
                       ))
nfi_file = os.path.join(candidate_path,
        candidate_file_name.format(
            task=task
            , data_type=data_type
            , model_version=nfi_model_version
            , model_for_eval=model_for_eval
            , topn=nfi_topn
                   ))
covisit_file = os.path.join(candidate_path,
        candidate_file_name.format(
            task=task
            , data_type=data_type
            , model_version=covisit_model_version
            , model_for_eval=model_for_eval
            , topn=covisit_topn
                   ))
# w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM).collect().sort('session_id')#.with_columns(pl.col('prev_items').apply(str2list))
# nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM).collect().sort('session_id')#.with_columns(pl.col('prev_items').apply(str2list))
# nfi_pl = pl.scan_parquet(nfi_file, n_rows=SAMPLE_NUM).collect().sort('session_id')#.with_columns(pl.col('prev_items').apply(str2list))
# covisit_pl = pl.scan_parquet(covisit_file, n_rows=SAMPLE_NUM).collect().sort('session_id')#.with_columns(pl.col('prev_items').apply(str2list))
dir_lst = []
pl_lst = []
for file_name in [w2v_file, nic_file, nfi_file, covisit_file]:
    data_pl = pl.scan_parquet(file_name, n_rows=SAMPLE_NUM).collect().sort('session_id')
    dir_name = file_name.replace('.parquet', '')
    print(dir_name)
    dir_lst.append(dir_name)
    pl_lst.append(data_pl)
    if not os.path.isdir(dir_name):
        os.mkdir(dir_name)
assert pl_lst[0].shape[0] == pl_lst[1].shape[0] == pl_lst[2].shape[0] == pl_lst[3].shape[0], f"{[ele.shape for ele in pl_lst]}"
total_len = pl_lst[0].shape[0]

# rows_per_part = int(total_len/parts)

# for dir_path, data_pl in zip(dir_lst, pl_lst):
#     print(dir_path)
#     for idx in tqdm(range(parts)):
#         begin = idx*rows_per_part
#         if idx != parts-1:
#             end = (idx+1)*rows_per_part
#         else:
#             end = total_len
#         part_pl = data_pl[begin:end :]
#         file_name = os.path.join(dir_path, f"part_{idx+1}.parquet")
#         print(f'{begin} - {end} -> {file_name}')
#         part_pl.write_parquet(file_name)